# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [4]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [5]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [30]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'datasets_fin/'

# model config
model_name = 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 256
LR = 1e-3
EPOCHS = 30
BATCH_SIZE = 32
num_workers = 0

In [63]:
df = pd.read_csv("../data/train.csv")
print(df)

                        ID  target
0     002f99746285dfdd.jpg      16
1     008ccd231e1fea5d.jpg      10
2     008f5911bfda7695.jpg      10
3     009235e4c9c07af5.jpg       4
4     00b2f44967580c74.jpg      16
...                    ...     ...
1565  fed9e9ec4a77bc06.jpg       4
1566  feeade617aa68c45.jpg       7
1567  ff51dd281a8423f1.jpg      11
1568  ff8a6a251ce51c95.jpg       5
1569  ffc22136f958deb1.jpg       9

[1570 rows x 2 columns]


데이터 증강

In [ ]:
import cv2
import pandas as pd
from tqdm import tqdm
import albumentations as A

# 원본 폴더와 증강 이미지 저장 폴더
input_dir = "../data/train"
output_dir = "../data/train_aug"
os.makedirs(output_dir, exist_ok=True)

# CSV 불러오기
df = pd.read_csv("../data/train.csv")

# 새 CSV 기록용
aug_data = []

# Albumentations transform 정의
augment = A.Compose([
    A.Rotate(limit=180, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(p=0.5)
])

for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_name = row['ID']
    label = row['target']
    img_path = os.path.join(input_dir, img_name)
    
    image = cv2.imread(img_path)
    
    # 원본 저장 및 CSV 기록
    cv2.imwrite(os.path.join(output_dir, img_name), image)
    aug_data.append({"filename": img_name, "label": label})

    # 증강 이미지 생성 (예: 2개)
    for i in range(2):
        augmented = augment(image=image)["image"]
        new_name = f"{os.path.splitext(img_name)[0]}_aug{i+1}.jpg"
        cv2.imwrite(os.path.join(output_dir, new_name), augmented)
        aug_data.append({"filename": new_name, "label": label})

# 새 CSV 저장
aug_df = pd.DataFrame(aug_data)
aug_df.to_csv("../data/aug_train.csv", index=False)

print("✅ 데이터 증강 및 CSV 생성 완료!")

100%|██████████| 1570/1570 [00:26<00:00, 58.91it/s]

✅ 데이터 증강 및 CSV 생성 완료!


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [ ]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    #A.Rotate(limit=180, p=0.5),  # ✅ Albumentations용 회전
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

증강 미사용

In [ ]:
# Dataset 정의
trn_dataset = ImageDataset(
    "../data/train.csv",
    "../data/train/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "../data/sample_submission.csv",
    "../data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

1570 3140


증강사용

In [76]:
# Dataset 정의
trn_dataset = ImageDataset(
    "../data/aug_train.csv",
    "../data/train_aug/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "../data/sample_submission.csv",
    "../data//test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

4710 3140


In [77]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [78]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [86]:
EPOCHS = 30
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 0.0001: 100%|██████████| 148/148 [00:24<00:00,  6.01it/s]


train_loss: 0.0081
train_acc: 0.9979
train_f1: 0.9978
epoch: 0.0000



Loss: 0.0152: 100%|██████████| 148/148 [00:24<00:00,  6.00it/s]


train_loss: 0.0020
train_acc: 1.0000
train_f1: 1.0000
epoch: 1.0000



Loss: 0.0027: 100%|██████████| 148/148 [00:24<00:00,  5.97it/s]


...

train_loss: 0.0006
train_acc: 1.0000
train_f1: 1.0000
epoch: 3.0000



Loss: 0.0004: 100%|██████████| 148/148 [00:24<00:00,  6.05it/s]


train_loss: 0.0003
train_acc: 1.0000
train_f1: 1.0000
epoch: 4.0000



Loss: 0.0001:  51%|█████     | 75/148 [00:12<00:12,  5.89it/s]


KeyboardInterrupt: 

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [87]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:14<00:00,  7.07it/s]


In [88]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [89]:
sample_submission_df = pd.read_csv("../data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [90]:
pred_df.to_csv("pred.csv", index=False)

In [91]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,6
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,6
4,00901f504008d884.jpg,2
